# Main Comparison Plots

In [ ]:
!pip install jsonlines
!ls
from preprocess import read_jsonlines_file, average_results
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import glob
import json
import pandas as pd

1m_output.jsonl  rpi4_cpus4.csv
docker_run.sh	 switching_result_assine2022a.json
dynamic.pdf	 switching_result_assine2022b.json
output.jsonl	 switching_result_lee2021.json
plots.ipynb	 switching_result_matsubara2022.json
preprocess.py


In [ ]:
sota_data = pd.read_csv("rpi4_cpus4.csv")

In [ ]:
def plot(ax, name, color='blue', marker='o', configurable=False):
    a = df[df.index.str.contains(name)]
    bw = a['KB'].values
    latency = a['ms'].values
    mAP = a['mAP'].values
    
    z = 80*(mAP**1.1)
    z_label = ["{:2.1f}".format(m) for m in mAP]
    
    vmin=10.0
    vmax=40.0
    
    if configurable:
         axi = ax.scatter(latency, bw, s=z, marker=marker, alpha=1.0, 
                    label=name, #linewidth=2,
#                     edgecolor='black',
                    c=mAP, cmap='viridis', vmin=vmin, vmax=vmax)
    else:
        axi = ax.scatter(latency, bw, s=z, marker=marker, alpha=1.0,
                   label=name,
#                    edgecolor='black',
                   c=mAP, cmap='viridis', vmin=vmin, vmax=vmax)
    for i in range(len(mAP)):
        ax.annotate(z_label[i], ( latency[i], bw[i]), ha='center', va='center', fontsize=18, color='black')

    return axi

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)
ax.set_xlabel('compute time (ms)')
ax.set_ylabel('bandwidth (KB)')
ax.set_xscale('log')
ax.set_yscale('log')

# Hide the right and top spines
ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)

f = "literature_comparison/literature.csv"
df.columns = df.columns.str.replace(' ', '')
plot(ax, "lee2021", color='cyan', marker='D')
plot(ax, "ours", color='red', marker="s", configurable=True)
plot(ax, "assine2022", color="#A8127D", marker="o", configurable=True)
axi = plot(ax, "matsubara2022", color='blue', marker="p")




# ADD ARROWS
bbox_props = dict(boxstyle="rarrow", fc="#0b5394", ec="b", lw=0)
t = ax.text(700, 2e1, "Inverse Tradeoff", ha="center", va="center", rotation=-45,
            size=24, color="white",
            bbox=bbox_props)
bbox_props = dict(boxstyle="larrow", fc="#a64d79", ec="b", lw=0)
t = ax.text(300, 7, "Direct Tradeoff", ha="center", va="center", rotation=25,
            size=24, color="white",
            bbox=bbox_props)

# LEGEND
legend_properties = {'weight':'normal', 'size': 24}
lgnd = ax.legend(fontsize=15,  prop=legend_properties, 
                 loc='upper left')
for i in range(4):
    lgnd.legendHandles[i]._sizes = [1e3]
#     lgnd.legendHandles[i].set_color('black')
#     lgnd.legendHandles[i].set_facecolor('white')

ax.set_xlim(150, 3.5e3)
ax.set_ylim(3, 1e3)
fig.colorbar(axi)

ax.set_xticks([2e2, 5e2, 1e3, 2e3])
ax.get_xaxis().set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))
plt.subplots_adjust(top = 0.97, bottom = .08, right = 1.1, left = .06, hspace = 0, wspace = 0.15)
plt.savefig("graph_sota.pdf", dpi=500)

# Switching Time Chart

In [ ]:

file_list = glob.glob("switching*")

columns=['name', 'num_configs', "ram(MB)", "disk_load(ms)", "warmup(ms)"]
num_configs_dict = {
    "assine2022b": 16,
    "assine2022a": 12,
    "matsubara2022": 4,
    "lee2021": 5,
}

rows = []

for file in file_list:
    with open(file, "r") as fp:
        data = json.load(fp)
        name = file.split('switching_result_')[1].split(".json")[0]
        num_configs = num_configs_dict[name]
        ram = data["ram"]*1000
        disk_load = sum([l["disk_load"] for l in data["lists"]])
        warmup = sum([l["warmup"] for l in data["lists"]])
        row = [name, num_configs, ram, disk_load, warmup]
        rows.append(row)
        
    

df = pd.DataFrame(rows, columns=columns)
df = df.set_index("name")
df = df.round(1)
print(df)
# print(df.to_latex())

## Dynamic Behaviour


In [ ]:


results = read_jsonlines_file('output.jsonl')
fig, ax = plt.subplots(5, 1, figsize=(5, 10), dpi=200)


for a in fig.get_axes():
    a.label_outer()
    
p0 = ax[0].scatter(time, e2e, s=8, color='r', label="E2E Latency")
# p0 = ax[0].plot(time, e2e, "r-", linewidth=0.7)
p0 = ax[0].plot(results["time"], results["deadline"], "r-")
p1 = ax[1].plot(results["time"], results["bw"], "b-", label="Bandwidth(kB/s)")
p2 = ax[2].plot(results["time"], results["mAP"], "g-", label="mAP")
p3 = ax[3].plot(results["time"], results["compute"], linewidth=0.7, color="black", label="mAP")
p4 = ax[4].plot(results["time"], results["bits"], linewidth=0.7, color="black", label="mAP")



ax[0].set_ylabel("E2E Latency (ms)")
ax[1].set_ylabel("Bandwidth (kB/s)")
ax[2].set_ylabel("mAP")
ax[3].set_ylabel("Ensemble Size")
ax[4].set_ylabel("Bits")
ax[4].set_xlabel("Seconds")

plt.savefig("dynamic.pdf")

# Dynamic - Fixed Distance

In [ ]:
import numpy as np
import jsonlines
ignore_start = 5
def get_mean(key):
    with jsonlines.open('1m_output.jsonl') as reader:
        values = [obj[key] for obj in reader]
    values = values[ignore_start:]
    return np.mean(values)
    

bw = get_mean("bw")
model_time = get_mean("model_time")*1000
e2e = get_mean("e2e")
mAP = get_mean("map")
deadline = get_mean("deadline")


width =0.3
data = [e2e-model_time, model_time]
print(data)
plt.bar(np.arange(len(data)), data, width=width)
plt.show()


# data1 = [23,85, 72, 43, 52]
# data2 = [42, 35, 21, 16, 9]

# plt.bar(np.arange(len(data1)), data1, width=width)
# plt.bar(np.arange(len(data2))+ width, data2, width=width)
# plt.show()

# Fixed Mode - Fixed Distance

In [ ]:
modes = [11, 12, 13, 14, 21, 22, 23, 24, 31, 32, 33, 34, 41, 42, 43, 44]

df = pd.Dataframe()

for mode in modes:
    results = read_jsonlines_file(f'output_static_{mode}.jsonl')
    results = average_results(results)
    df = df.append(results, ignore_index=True)
    
print(df)